#  1. Load and Prepare the Data

In [ ]:
import pandas as pd

# Load your data
df = pd.read_csv('../data/yfinance_data/AAPL_historical_data.csv', parse_dates=['Date'])
df.set_index('Date', inplace=True)
df = df.sort_index()

# Ensure required columns are present
required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
assert all(col in df.columns for col in required_cols), "Missing required columns"
df.head()
